In [23]:
from __future__ import division
import pandas as pd
import numpy as np
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,roc_auc_score
from datetime import datetime
from dateutil.relativedelta import relativedelta as rd

In [24]:
ini =datetime.now()

In [25]:
df = pd.read_csv('Datos/TSC_churn_telecom.txt')

In [26]:
df.head()

,V_ACCT_LGHT,C_IP,C_VMP,V_N_VM_MESS,V_T_DAY_CHG,V_T_EVE_CALLS,V_T_EVE_CHG,V_T_NIG_CALLS,V_T_NIG_CHG,V_T_INT_CALLS,V_T_INT_CHG,V_CS_CALLS,TARGET,ID
0,128,No,Yes,25,45.07,99,16.78,91,11.01,3,2.70,1,0,1
1,107,No,Yes,26,27.47,103,16.62,103,11.45,3,3.70,1,0,2
2,137,No,No,0,41.38,110,10.30,104,7.32,5,3.29,0,0,3
3,65,No,No,0,21.95,83,19.42,111,9.40,6,3.43,4,1,11
4,168,No,No,0,21.90,71,8.92,128,6.35,2,3.02,1,0,13


In [27]:
var_cont = [v for v in df.columns if v[:2]=='V_']
var_disc = [v for v in df.columns if v[:2]=='C_']

In [28]:
print var_cont
print var_disc

['V_ACCT_LGHT', 'V_N_VM_MESS', 'V_T_DAY_CHG', 'V_T_EVE_CALLS', 'V_T_EVE_CHG', 'V_T_NIG_CALLS', 'V_T_NIG_CHG', 'V_T_INT_CALLS', 'V_T_INT_CHG', 'V_CS_CALLS']
['C_IP', 'C_VMP']


In [29]:
def woe_code(df,var_disc):
    for v in var_disc:
        aux = df[[v,'ID','TARGET']].copy()
        aux = aux.pivot_table(aggfunc='count',
                              columns='TARGET',
                              fill_value=0,
                              index=v)
        aux = aux['ID']
        aux.reset_index(inplace=True)
        aux

        for i in range(2):
            aux[i]/=aux[i].sum()

        aux['woe'] = np.log(aux[0]/aux[1])
        print "IV variable %s = %.3f " %(v,((aux[0]-aux[1])*aux['woe']).sum())
        df = df.merge(aux[[v,'woe']],on=v,how='inner')
        #df.drop(v,axis=1,inplace=True)
        df.rename(columns={'woe':'W_'+v},inplace=True)
    return df

In [30]:
df = woe_code(df,var_disc)

IV variable C_IP = 0.360 
IV variable C_VMP = 0.075 


In [31]:
df.head()

,V_ACCT_LGHT,C_IP,C_VMP,V_N_VM_MESS,V_T_DAY_CHG,V_T_EVE_CALLS,V_T_EVE_CHG,V_T_NIG_CALLS,V_T_NIG_CHG,V_T_INT_CALLS,V_T_INT_CHG,V_CS_CALLS,TARGET,ID,W_C_IP,W_C_VMP
0,128,No,Yes,25,45.07,99,16.78,91,11.01,3,2.70,1,0,1,0.258836,0.510386
1,107,No,Yes,26,27.47,103,16.62,103,11.45,3,3.70,1,0,2,0.258836,0.510386
2,85,No,Yes,27,33.39,90,23.88,75,4.02,4,3.73,1,0,17,0.258836,0.510386
3,76,No,Yes,33,32.25,65,18.09,108,7.46,5,2.70,1,0,19,0.258836,0.510386
4,57,No,Yes,39,36.21,112,16.24,115,8.22,3,2.57,0,0,27,0.258836,0.510386


In [35]:
for v in var_cont:
    df.rename(columns={v:'x_%s'%v},inplace=True)
    df[v] = pd.cut(df['x_%s'%v],bins=5).astype(str)

In [37]:
df.head()

,x_V_ACCT_LGHT,C_IP,C_VMP,x_V_N_VM_MESS,x_V_T_DAY_CHG,x_V_T_EVE_CALLS,x_V_T_EVE_CHG,x_V_T_NIG_CALLS,x_V_T_NIG_CHG,x_V_T_INT_CALLS,...,V_ACCT_LGHT,V_N_VM_MESS,V_T_DAY_CHG,V_T_EVE_CALLS,V_T_EVE_CHG,V_T_NIG_CALLS,V_T_NIG_CHG,V_T_INT_CALLS,V_T_INT_CHG,V_CS_CALLS
0,128,No,Yes,25,45.07,99,16.78,91,11.01,3,...,"(121.8, 158.4]","(17.2, 25.8]","(43.266, 51.88]","(90.6, 109.4]","(14.892, 18.968]","(75.0, 93.0]","(10.008, 12.154]","(0.988, 3.4]","(2.338, 3.062]","(-0.006, 1.2]"
1,107,No,Yes,26,27.47,103,16.62,103,11.45,3,...,"(85.2, 121.8]","(25.8, 34.4]","(26.038, 34.652]","(90.6, 109.4]","(14.892, 18.968]","(93.0, 111.0]","(10.008, 12.154]","(0.988, 3.4]","(3.062, 3.786]","(-0.006, 1.2]"
2,85,No,Yes,27,33.39,90,23.88,75,4.02,4,...,"(48.6, 85.2]","(25.8, 34.4]","(26.038, 34.652]","(71.8, 90.6]","(23.044, 27.12]","(56.91, 75.0]","(3.559, 5.716]","(3.4, 5.8]","(3.062, 3.786]","(-0.006, 1.2]"
3,76,No,Yes,33,32.25,65,18.09,108,7.46,5,...,"(48.6, 85.2]","(25.8, 34.4]","(26.038, 34.652]","(52.906, 71.8]","(14.892, 18.968]","(93.0, 111.0]","(5.716, 7.862]","(3.4, 5.8]","(2.338, 3.062]","(-0.006, 1.2]"
4,57,No,Yes,39,36.21,112,16.24,115,8.22,3,...,"(48.6, 85.2]","(34.4, 43.0]","(34.652, 43.266]","(109.4, 128.2]","(14.892, 18.968]","(111.0, 129.0]","(7.862, 10.008]","(0.988, 3.4]","(2.338, 3.062]","(-0.006, 1.2]"


In [38]:
df = woe_code(df,var_cont)

IV variable V_ACCT_LGHT = 0.006 
IV variable V_N_VM_MESS = 0.109 
IV variable V_T_DAY_CHG = 0.553 
IV variable V_T_EVE_CALLS = 0.007 
IV variable V_T_EVE_CHG = 0.070 
IV variable V_T_NIG_CALLS = 0.014 
IV variable V_T_NIG_CHG = 0.016 
IV variable V_T_INT_CALLS = 0.067 
IV variable V_T_INT_CHG = 0.033 
IV variable V_CS_CALLS = 0.564 


In [39]:
df.drop(var_cont,axis=1,inplace=True)

In [42]:
df.rename(columns=dict(zip(['x_%s'%v for v in var_cont],var_cont)),inplace=True)

In [43]:
df.head()

,V_ACCT_LGHT,C_IP,C_VMP,V_N_VM_MESS,V_T_DAY_CHG,V_T_EVE_CALLS,V_T_EVE_CHG,V_T_NIG_CALLS,V_T_NIG_CHG,V_T_INT_CALLS,...,W_V_ACCT_LGHT,W_V_N_VM_MESS,W_V_T_DAY_CHG,W_V_T_EVE_CALLS,W_V_T_EVE_CHG,W_V_T_NIG_CALLS,W_V_T_NIG_CHG,W_V_T_INT_CALLS,W_V_T_INT_CHG,W_V_CS_CALLS
0,128,No,Yes,25,45.07,99,16.78,91,11.01,3,...,0.061033,1.219037,-1.864202,0.068283,0.065696,-0.089148,-0.05179,-0.29035,0.007263,0.295956
1,156,No,No,0,29.67,116,16.78,86,10.73,2,...,0.061033,-0.147174,0.561314,-0.053201,0.065696,-0.089148,-0.05179,-0.29035,0.007263,0.295956
2,57,No,No,0,14.60,127,16.48,93,10.42,2,...,0.053118,-0.147174,0.178109,-0.053201,0.065696,-0.089148,-0.05179,-0.29035,0.007263,0.295956
3,58,No,Yes,20,33.07,89,18.16,92,10.65,2,...,0.053118,1.219037,0.561314,0.031576,0.065696,-0.089148,-0.05179,-0.29035,0.007263,0.295956
4,149,No,No,0,31.89,81,17.12,79,11.89,1,...,0.061033,-0.147174,0.561314,0.031576,0.065696,-0.089148,-0.05179,-0.29035,0.007263,0.295956


In [44]:
pickle.dump(df,open('churn_con_woe','wb'))

In [15]:
model = LogisticRegression()

In [16]:
var_woe = [v for v in df.columns if v[:2]=='W_']

In [17]:
X = df[var_woe]
y = df['TARGET']

In [18]:
Xt,Xv,yt,yv = train_test_split(X,y,train_size=0.7)

In [19]:
model.fit(Xt,yt)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [20]:
def metricas(model,Xt,Xv,yt,yv):
    print "ROC train:%.3f | ROC test:%.3f " %(roc_auc_score(y_true=yt,y_score=model.predict_proba(Xt)[:,1]),roc_auc_score(y_true=yv,y_score=model.predict_proba(Xv)[:,1]))
    print "ACC train:%.3f | ACC test:%.3f " %(accuracy_score(y_true=yt,y_pred=model.predict(Xt)),accuracy_score(y_true=yv,y_pred=model.predict(Xv)))

In [21]:
metricas(model,Xt,Xv,yt,yv )

ROC train:0.874 | ROC test:0.856 
ACC train:0.886 | ACC test:0.899 


In [22]:
fin =datetime.now()
print rd(fin,ini)

relativedelta(seconds=+1, microseconds=+349594)
